In [1]:
import pandas as pd
import calendar
from consts import *
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error


In [2]:
real_monthly_production = pd.read_csv(MONTHLY_REAL_PROD)
real_monthly_production[MONTH] = real_monthly_production[MONTH]\
    .apply(lambda month: calendar.month_abbr[month] + "_" +  KWH)
real_monthly_production.head()


,etak_id,pv_area_m2,source,address,monthly_production_kwh,month,year
0,719155,195.82,540121820,Tiigi 19,1655.56,Mar_kwh,2021
1,719155,195.82,540121820,Tiigi 19,3691.71,Apr_kwh,2021
2,719155,195.82,540121820,Tiigi 19,4160.20,May_kwh,2021
3,719155,195.82,540121820,Tiigi 19,5914.39,Jun_kwh,2021
4,719155,195.82,540121820,Tiigi 19,5766.83,Jul_kwh,2021


In [3]:
columns_names = [ETAK_ID, SOURCE, ADDRESS, YEAR, PV_AREA_M2]
real_monthly_production_pivot = real_monthly_production.pivot(
    index=columns_names, columns=[MONTH], 
    values=["monthly_production_kwh"]
)
real_monthly_production_pivot.columns = real_monthly_production_pivot.columns.droplevel()
real_monthly_production_pivot.head(4)

month                                      Apr_kwh  Aug_kwh  Dec_kwh  Feb_kwh  \
etak_id source    address year pv_area_m2                                       
713413  547236060 Turu 3  2020 165.44          NaN      NaN   109.30      NaN   
                          2021 165.44      2896.44  2565.60    17.27   109.39   
                          2022 165.44      3373.03  2701.75      NaN   546.71   
715116  547236058 Turu 9  2020 147.85          NaN      NaN    88.23      NaN   

month                                      Jan_kwh  Jul_kwh  Jun_kwh  Mar_kwh  \
etak_id source    address year pv_area_m2                                       
713413  547236060 Turu 3  2020 165.44          NaN      NaN      NaN      NaN   
                          2021 165.44        14.87  4452.62  4817.43  1360.82   
                          2022 165.44        99.40  4102.70  4439.18  2507.70   
715116  547236058 Turu 9  2020 147.85          NaN      NaN      NaN      NaN   

month                                      May_kwh  Nov_kwh  Oct_kwh  Sep_kwh  
etak_id source    address year pv_area_m2                                      
713413  547236060 Turu 3  2020 165.44          NaN   317.82      NaN      NaN  
                          2021 165.44      3353.91   325.46  1091.42      NaN  
                          2022 165.44      3937.53   171.81   913.92  1987.08  
715116  547236058 Turu 9  2020 147.85          NaN   292.29      NaN      NaN

In [4]:
monthly_energy_production = real_monthly_production_pivot.rename_axis(None, axis=1)
monthly_energy_production= monthly_energy_production.reset_index()
monthly_energy_production = monthly_energy_production.reindex(
    columns_names + MONTHS_ABBR_UNITS,
    axis=1
)
monthly_energy_production.head(4)

,etak_id,source,address,year,pv_area_m2,Jan_kwh,Feb_kwh,Mar_kwh,Apr_kwh,May_kwh,Jun_kwh,Jul_kwh,Aug_kwh,Sep_kwh,Oct_kwh,Nov_kwh,Dec_kwh
0,713413,547236060,Turu 3,2020,165.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,317.82,109.30
1,713413,547236060,Turu 3,2021,165.44,14.87,109.39,1360.82,2896.44,3353.91,4817.43,4452.62,2565.60,NaN,1091.42,325.46,17.27
2,713413,547236060,Turu 3,2022,165.44,99.40,546.71,2507.70,3373.03,3937.53,4439.18,4102.70,2701.75,1987.08,913.92,171.81,NaN
3,715116,547236058,Turu 9,2020,147.85,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,292.29,88.23


In [5]:
monthly_energy_production[MONTHS_ABBR_UNITS_PV_M2] = monthly_energy_production[MONTHS_ABBR_UNITS]\
    .divide(monthly_energy_production[PV_AREA_M2], axis=0)

In [6]:
monthly_energy_production.to_csv(DATA_DIR.joinpath("real_monthly_prod.csv"), index=False)

### Monthly comparison

In [24]:
estimated_production_by_building = pd.read_csv(BUILDINGS_PATH)
suffixes = ["_r", "_e"]

In [25]:
def merge_dataframes(year, real_production_df, columns_real_prod, columns_estimated_prod):
    production_for_year = real_production_df[real_production_df[YEAR] == year]
    return production_for_year[columns_real_prod].merge(
        estimated_production_by_building[columns_estimated_prod], 
        how="left", 
        on=ETAK_ID, 
        suffixes=suffixes
    )

In [29]:
monthly_real_prod_columns = [ETAK_ID, ADDRESS, YEAR, PV_AREA_M2] + MONTHS_ABBR_UNITS_PV_M2
monthly_est_prod_columns = [ETAK_ID, PV_AREA_M2] + MONTHS_ABBR_UNITS_PV_M2

merged_monthly_production_2022 = merge_dataframes(
    2022,
    monthly_energy_production,
    monthly_real_prod_columns,
    monthly_est_prod_columns
)


In [30]:
seasons = ["Winter", "Spring", "Summer", "Autumn"]

labels = []
for abbr in [MONTHS_ABBR_UNITS_PV_M2[-1]] + MONTHS_ABBR_UNITS_PV_M2[0:-1]:
    labels.append(abbr + suffixes[0])
    labels.append(abbr + suffixes[1])

pv_lables = [PV_AREA_M2 + suffixes[0], PV_AREA_M2 + suffixes[1]]

In [32]:
def construct_comparison_csv():
    for i in range(len(seasons)):
        with open(f"{DATA_DIR.joinpath('output/prod_comparison.csv')}", 'a') as file:
            file.write(f"\n{seasons[i]} 2022\n")
        columns = [ETAK_ID, ADDRESS, YEAR] + pv_lables + labels[i*6:(i+1)*6]
        merged_monthly_production_2022[columns].to_csv(
            DATA_DIR.joinpath('output/prod_comparison.csv'),
            mode='a',
            header=True,
            index=False,
            na_rep="NaN"
        )
            
construct_comparison_csv()

In [26]:
merged_monthly_production_2022 = merged_monthly_production_2022[labels].fillna(merged_monthly_production_2022[labels].mean())

In [27]:
for i in range(len(seasons)):
    columns = labels[i*6:(i+1)*6]
    seasonal_real_prod = merged_monthly_production_2022[columns[0]] + merged_monthly_production_2022[columns[2]] + merged_monthly_production_2022[columns[4]]
    seasonal_estimated_prod = merged_monthly_production_2022[columns[1]] + merged_monthly_production_2022[columns[3]] + merged_monthly_production_2022[columns[5]]
    try:
        print(f"MAPE for {seasons[i]}: {mean_absolute_percentage_error(seasonal_real_prod, seasonal_estimated_prod) * 100:.2f}%")
    except Exception as e:
        print(f"Cannot calculate MAPE for season {seasons[i]} due to the error {e}") 

Cannot calculate MAPE for season Winter due to the error Input contains NaN.
MAPE for Spring: 15.27%
MAPE for Summer: 19.16%
MAPE for Autumn: 31.93%


In [118]:
winter_real = merged_monthly_production_2022[labels[2]] + merged_monthly_production_2022[labels[4]]
winter_estimated = merged_monthly_production_2022[labels[3]] + merged_monthly_production_2022[labels[5]]
print(f"MAPE for Winter: {mean_absolute_percentage_error(winter_real, winter_estimated) * 100:.2f}%")

MAPE for Winter: 119.08%


### Annual comparison

In [19]:
real_annual_energy_production = pd.read_csv(ANNUAL_REAL_PROD)
real_annual_energy_production[ANNUAL_KWH_PV_M2] = real_annual_energy_production[ANNUAL_KWH]\
    .divide(real_annual_energy_production[PV_AREA_M2], axis=0)
real_annual_energy_production.head()

,etak_id,pv_area_m2,source,address,annual_kwh,year,annual_kwh/pv_m2
0,719155,195.82,540121820,Tiigi 19,29183.68,2021,149.033194
1,719155,195.82,540121820,Tiigi 19,33144.35,2022,169.259269
2,716647,182.69,547236018,Tähe 2,27484.60,2021,150.443921
3,716647,182.69,547236018,Tähe 2,30381.91,2022,166.303082
4,720043,146.65,547236054,Aleksandri 3,23177.16,2022,158.044050


In [20]:
annual_real_prod_columns = [ETAK_ID, ADDRESS, YEAR, PV_AREA_M2, ANNUAL_KWH_PV_M2] + [ANNUAL_KWH]
annual_est_prod_columns = [ETAK_ID, PV_AREA_M2, ANNUAL_KWH_PV_M2] + [ANNUAL_KWH]

annual_pv_m2_merged_2022 = merge_dataframes(
    2022,
    real_annual_energy_production,
    annual_real_prod_columns, 
    annual_est_prod_columns
)

In [84]:
annual_pv_m2_merged_2022.round(2)

,etak_id,address,year,pv_area_m2_r,annual_kwh/pv_m2_r,annual_kwh_r,pv_area_m2_e,annual_kwh/pv_m2_e,annual_kwh_e
0,719155,Tiigi 19,2022,195.82,169.26,33144.35,453.98,187.66,85193.55
1,716647,Tähe 2,2022,182.69,166.30,30381.91,461.86,199.47,92127.33
2,720043,Aleksandri 3,2022,146.65,158.04,23177.16,468.83,175.85,82442.36
3,715116,Turu 9,2022,147.85,160.65,23752.37,482.05,176.14,84910.44
4,713413,Turu 3,2022,165.44,149.79,24780.80,477.84,211.86,101236.31
5,720467,Pepleri 3,2022,139.91,142.09,19879.62,428.36,176.30,75521.34
6,720239,Pepleri 12,2022,171.77,155.80,26761.90,727.41,176.54,128415.56
7,720448,J.Kuperjanovi 2,2022,193.61,149.63,28969.60,583.53,178.12,103937.58
8,719935,Lutsu 16,2022,187.23,107.96,20213.32,427.43,177.57,75897.73
9,720853,Tiigi 21,2022,161.09,144.85,23333.76,423.89,175.03,74192.33


In [21]:
mean_squared_error(annual_pv_m2_merged_2022[ANNUAL_KWH_PV_M2 + "_r"],
                   annual_pv_m2_merged_2022[ANNUAL_KWH_PV_M2 + "_e"], squared=False)


38.085192125644674

In [22]:
mean_absolute_percentage_error(annual_pv_m2_merged_2022[ANNUAL_KWH_PV_M2 + "_r"], annual_pv_m2_merged_2022[ANNUAL_KWH_PV_M2 + "_e"])

0.23610090851975393

In [23]:
annual_pv_m2_merged_2022.to_csv(DATA_DIR.joinpath("output/annual_prod_comparison_2022.csv"), index=False)